<a href="https://colab.research.google.com/github/uucad/aigc_LLM_engineering/blob/main/chatpdf_2_query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/chatglm
!git clone https://github.com/cystanford/aigc_LLM_engineering
%cd /content/drive/MyDrive/chatglm/aigc_LLM_engineering/3-ChatPDF实现/

/content/drive/MyDrive/chatglm
fatal: destination path 'aigc_LLM_engineering' already exists and is not an empty directory.
/content/drive/MyDrive/chatglm/aigc_LLM_engineering/3-ChatPDF实现


In [4]:
!pip install langchain
!pip install openai
!pip install chromadb
!pip install tiktoken
!pip install unstructured
!pip install pdf2image

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 4.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 3.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.6/123.6 kB 3

In [5]:
import os
# os.environ["OPENAI_API_KEY"] = "your-api-key-here"

In [6]:
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader
from langchain.chains.question_answering import load_qa_chain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

# os.environ["OPENAI_API_KEY"] = "{your-api-key}"

global retriever
def load_embedding():
    global retriever
    # 使用OpenAI Embedding
    embedding = OpenAIEmbeddings()
    # 指定了persist_directory, 所以在本地会有db文件夹，保存持久化结果
    vectordb = Chroma(persist_directory='db', embedding_function=embedding)
    # 检索Top4
    retriever = vectordb.as_retriever(search_kwargs={"k": 4})

# 基于query进行问答
def prompt(query):
    prompt_template = """
    请注意：请谨慎评估query与提示的Context信息的相关性，
    只根据本段输入文字信息的内容进行回答，如果query与提供的材料无关，请回答"我不知道"，
    另外也不要回答无关答案：
    Context: {context}
    Question: {question}
    Answer:"""
    PROMPT = PromptTemplate(
        input_variables=["context", "question"],
        template=prompt_template
    )
    # 从向量数据库中，检索出Top5相似文档
    docs = retriever.get_relevant_documents(query)
    print('docs=', docs)
    print('len(docs)=', len(docs))
    
    # 基于docs来prompt，返回你想要的内容
    chain = load_qa_chain(ChatOpenAI(temperature=0), chain_type="stuff", prompt=PROMPT)
    result = chain({"input_documents": docs, "question": query}, return_only_outputs=True)
    return result['output_text']

In [7]:
# 加载embedding
load_embedding()
print('hhhhh')
query = "桃园三结义都有谁"
#query = "曹操都有哪些部将"

print("Query:" + query + '\nAnswer:' + prompt(query) + '\n')

hhhhh
docs= []
len(docs)= 0
Query:桃园三结义都有谁
Answer:刘备、关羽、张飞。

